# Covert imglab format to PASCAL VOC format

This notebook is designed to give insights how imglab formart used in `dlib` library can be converted to PASCAL VOC format in order to use TensorFlow Object Detection API

Data directory structure:

```
.
|--Annotations
| |
| -xmls
|--dataset1
|--dataset2
|--dataset1.xml
|--dataset2.xml
|--merged.xml
|--train.record-?????-of-00010
|--val.record-?????-of-00010
```

The `Annotations` folder will be the place where PASCAL VOC annotations will be written to.

The `train.record` and `val.record` are TFRecord files which can be used for the TensorFlow Object Detection API to train models.

## Import necessary packages and dependencies

In [ ]:
import os
import tensorflow as tf
import xml.etree.ElementTree as et

from PIL import Image
from pascal_voc_writer import Writer

In [ ]:
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('database', 'AutoLens',
                           'Database name')
tf.app.flags.DEFINE_string('imglab_xml', 'merged.xml',
                           'Path to the imglab annotation file')
tf.app.flags.DEFINE_string('xml_dir', 'Annotations/xmls',
                           'Path to PASCAL VOC xml files')

## Parsing XML

```xml
<name>imglab dataset</name>
<comment>Created by imglab tool.</comment>
<images>
  <image file='Images/1Img_1.jpg'>
    <box top='665' left='1366' width='187' height='141'>
      <label>label</label>
    </box>
    <box top='607' left='1113' width='54' height='70'>
      <label>label</label>
    </box>
    <box top='389' left='738' width='337' height='282' ignore='1'>
      <label>label</label>
    </box>
    <box top='413' left='1171' width='707' height='230' ignore='1'>
      <label>label</label>
    </box>
  </image>
</images>
```

## Finding elements

In [ ]:
def create_xml_annotation(database, imglab_file_path, xml_dir):
    if not os.path.isdir(xml_dir):
        os.makedirs(xml_dir)
    
    tree = et.parse('merged.xml')  
    root = tree.getroot()
    images = root.find('images')
    
    # Finds the first child with the 'images' tag
    for image in images.iter(tag='image'):
        image_path = image.attrib['file']
        folder = os.path.dirname(image_path)
        raw_filename = os.path.basename(image_path)
        filename, file_extension = os.path.splitext(raw_filename)

        img = Image.open(image_path)
        width, height = img.size

        writer = Writer(image_path, width, height, database=database)

        num_objects = 0
        for box in image:
            if 'ignore' in box.attrib.keys():
                continue
            num_objects += 1
            xmin = box.attrib['left']
            ymin = box.attrib['top']
            xmax = int(xmin) + int(box.attrib['width'])
            ymax = int(ymin) + int(box.attrib['height'])
            writer.addObject(
                name=box[0].text,
                xmin=xmin,
                ymin=ymin,
                xmax=xmax,
                ymax=ymax
            )
        if num_objects > 0:
            writer.save(os.path.join(FLAGS.xml_dir, f'{folder}_{filename}.xml'))


def main(unused_argv):
    create_xml_annotation(FLAGS.database, FLAGS.imglab_xml, FLAGS.xml_dir)

    
if __name__ == '__main__':
    tf.app.run()

## Convert the PASCAL VOC data to TFRecord

In [ ]:
import hashlib
import io
import logging
import os
import random
import re

import contextlib2
from lxml import etree
from PIL import Image
import numpy as np
import tensorflow as tf

from object_detection.dataset_tools import tf_record_creation_util
from object_detection.utils import dataset_util
from object_detection.utils import label_map_util


FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('annotations_dir', 'Annotations',
                           '(Relative) path to annotations directory.')
tf.app.flags.DEFINE_string('output_dir', '', 'Path to directory to output TFRecords.')
tf.app.flags.DEFINE_string('label_map_path', 'autolens_label_map.pbtxt',
                           'Path to label map proto')
tf.app.flags.DEFINE_boolean('ignore_difficult_instances', False, 'Whether to ignore '
                            'difficult instances')
tf.app.flags.DEFINE_integer('num_shards', 10, 'Number of TFRecord shards')

DATASETS = ['GTSDB_VOC', 'AutoLens']

In [ ]:
def dict_to_tf_example(data,
                       label_map_dict,
                       ignore_difficult_instances=False):
    """Convert XML derived dict to tf.Example proto.
    
    Notice that this function normalizes the bounding box coordinates provided
    by the raw data.
    
    Args:
        data: dict holding PASCAL XML fields for a single image (obtained by
            running dataset_util.recursive_parse_xml_to_dict)
        dataset_directory: Path to root directory holding PASCAL dataset
        label_map_dict: A map from string label names to integers ids.
        image_subdirectory: String specifying subdirectory within the
            PASCAL dataset directory holding the actual image data.
        ignore_difficult_instances: Whether to skip difficult instances in the
            dataset  (default: False).
    
    Returns:
        example: The converted tf.Example.
    
    Raises:
        ValueError: if the image pointed to by data['filename'] is not a valid JPEG
    """
    img_path = os.path.join(data['folder'], data['filename'])
    with tf.gfile.GFile(img_path, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    if image.format != 'JPEG':
        raise ValueError('Image format is not JPEG')
    key = hashlib.sha256(encoded_jpg).hexdigest()

    width = int(data['size']['width'])
    height = int(data['size']['height'])

    xmin = []
    ymin = []
    xmax = []
    ymax = []
    classes = []
    classes_text = []
    truncated = []
    poses = []
    difficult_obj = []
    if 'object' in data:
        for obj in data['object']:
            difficult = bool(int(obj['difficult']))
            if ignore_difficult_instances and difficult:
                continue

            difficult_obj.append(int(difficult))

            xmin.append(float(obj['bndbox']['xmin']) / width)
            ymin.append(float(obj['bndbox']['ymin']) / height)
            xmax.append(float(obj['bndbox']['xmax']) / width)
            ymax.append(float(obj['bndbox']['ymax']) / height)
            classes_text.append(obj['name'].encode('utf8'))
            classes.append(label_map_dict[obj['name']])
            truncated.append(int(obj['truncated']))
            poses.append(obj['pose'].encode('utf8'))

    example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(
            data['filename'].encode('utf8')),
        'image/source_id': dataset_util.bytes_feature(
            data['filename'].encode('utf8')),
        'image/key/sha256': dataset_util.bytes_feature(key.encode('utf8')),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmin),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmax),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymin),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymax),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
        'image/object/difficult': dataset_util.int64_list_feature(difficult_obj),
        'image/object/truncated': dataset_util.int64_list_feature(truncated),
        'image/object/view': dataset_util.bytes_list_feature(poses),
        }))
    
    return example

In [ ]:
def create_tf_record(output_filename,
                     num_shards,
                     label_map_dict,
                     annotations_dir,
                     examples):
    """Creates a TFRecord file from examples.
    Args:
      output_filename: Path to where output file is saved.
      num_shards: Number of shards for output file.
      label_map_dict: The label map dictionary.
      annotations_dir: Directory where annotation files are stored.
      examples: Examples to parse and save to tf record.
    """
    with contextlib2.ExitStack() as tf_record_close_stack:
        output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(
            tf_record_close_stack, output_filename, num_shards)
        for idx, example in enumerate(examples):
            if idx % 100 == 0:
                logging.info(f'On image {idx} of {len(examples)}')
            xml_path = os.path.join(annotations_dir, 'xmls', example + '.xml')
            
            if not os.path.exists(xml_path):
                logging.warning(f'Could not find {xml_path}, ignoring example.')
                continue
            with tf.gfile.GFile(xml_path, 'r') as fid:
                xml_str = fid.read()
            xml = etree.fromstring(xml_str)
            data = dataset_util.recursive_parse_xml_to_dict(xml)['annotation']

            try:
                tf_example = dict_to_tf_example(
                    data,
                    label_map_dict)
                if tf_example:
                    shard_idx = idx % num_shards
                    output_tfrecords[shard_idx].write(
                        tf_example.SerializeToString())
            except ValueError:
                logging.warning(f'Invalid example: {xml_path}, ignoring.')

In [ ]:
def main(unused_argv):
    if FLAGS.database not in DATASETS:
        raise ValueError(f'set must be in : {DATASETS}')
    
    data_dir = os.getcwd()
    label_map_dict = label_map_util.get_label_map_dict(FLAGS.label_map_path)
    
    logging.info(f'Reading from {FLAGS.database} dataset.')
    annotations_dir = os.path.join(data_dir, 'Annotations')
    examples_path = os.path.join(annotations_dir, 'trainval.txt')
    examples_list = dataset_util.read_examples_list(examples_path)
    
    # Test images are not included in the downloaded data set, so we shall perform
    # our own split.
    random.seed(42)
    random.shuffle(examples_list)
    num_examples = len(examples_list)
    num_train = int(0.7 * num_examples)
    train_examples = examples_list[:num_train]
    val_examples = examples_list[num_train:]
    logging.info('%d training and %d validation examples.',
        len(train_examples), len(val_examples))
    
    train_output_path = os.path.join(FLAGS.output_dir, 'train.record')
    val_output_path = os.path.join(FLAGS.output_dir, 'val.record')
    
    create_tf_record(
        train_output_path,
        FLAGS.num_shards,
        label_map_dict,
        annotations_dir,
        train_examples)
    create_tf_record(
        val_output_path,
        FLAGS.num_shards,
        label_map_dict,
        annotations_dir,
        val_examples)


if __name__ == '__main__':
    tf.app.run()